In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

import tensorflow as tf
from keras import layers
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras import backend as K
from keras.models import Model
from keras import initializers, regularizers
from keras.regularizers import l2
from keras_self_attention import SeqSelfAttention
import keras

Using TensorFlow backend.


In [2]:
def int_sentence(left_texts, right_texts, vocab_index):
    left_int = []
    right_int = []

    for i in range(len(left_texts)):
        left_etc = []
        right_etc = []
        for j in range(len(left_texts[i])):
            left_etc.append(vocab_index[left_texts[i][j]])
        for j in range(len(right_texts[i])):
            right_etc.append(vocab_index[right_texts[i][j]])
        left_int.append(left_etc)
        right_int.append(right_etc)
        
    return left_int, right_int

In [3]:
All = open('file_name_1', 'r', encoding='utf-8-sig')   # train, cv, test가 모두 있는 파일
train = open('file_name_2', 'r', encoding='utf-8-sig')
cv = open('file_name_3', 'r', encoding='utf-8-sig')
test = open('file_name_4', 'r', encoding='utf-8-sig')

In [4]:
all_sentence = []

for line in All:
    line = line.split('\t')
    all_sentence.append(line[0].split())
    all_sentence.append(line[1].split())

max_len = max([len(i) for i in all_sentence])

vocab = set()
for line in all_sentence:
    for word in line:
        vocab.add(word)

vocab_size = len(vocab)+1

vocab = sorted(list(vocab))

vocab_index = {}
for i in range(len(vocab)):
    vocab_index[vocab[i]] = len(vocab_index)+1

In [5]:
embed_model_dir = 'embedding_pretrain_file'

embedding_index = dict()

with open(embed_model_dir, 'r', encoding='utf-8-sig') as f:
    for line in f:
        # Exception to first line in word2vec model.
        if len(line.split()) == 2:
            continue
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vectors
    f.close()
    print('--- Finished make embedding index ---')
print('Found %s word vectors.' % len(embedding_index))

EMBEDDING_DIM = 300

embedded_matrix = np.zeros((len(vocab_index)+1, EMBEDDING_DIM))
embed_cnt = 0

for word, i in vocab_index.items():
    embedded_vector = embedding_index.get(word)
    if embedded_vector is not None:
        embedded_matrix[i] = embedded_vector
        embed_cnt += 1

print('Created Embedded Matrix: %s word vectors.' % embed_cnt)

--- Finished make embedding index ---
Found 12178 word vectors.
Created Embedded Matrix: 12178 word vectors.


In [6]:
embedded_layer = layers.Embedding(len(vocab_index)+1, EMBEDDING_DIM, 
    weights=[embedded_matrix], input_length=max_len, trainable=False)

train_left_sen = []
train_right_sen = []
train_label = []

for line in train:
    line = line.split('\t')
    train_left_sen.append(line[0].strip().split())
    train_right_sen.append(line[1].strip().split())
    train_label.append(float(line[2].strip()))

cv_left_sen = []
cv_right_sen = []
cv_label = []

for line in cv:
    line = line.split('\t')
    cv_left_sen.append(line[0].strip().split())
    cv_right_sen.append(line[1].strip().split())
    cv_label.append(float(line[2].strip()))

test_left_sen = []
test_right_sen = []
test_label = []

for line in test:
    line = line.split('\t')
    test_left_sen.append(line[0].strip().split())
    test_right_sen.append(line[1].strip().split())
    test_label.append(float(line[2].strip()))

train_left_int, train_right_int = int_sentence(train_left_sen, train_right_sen, vocab_index)
cv_left_int, cv_right_int = int_sentence(cv_left_sen, cv_right_sen, vocab_index)
test_left_int, test_right_int = int_sentence(test_left_sen, test_right_sen, vocab_index)

train_left = pad_sequences(train_left_int, padding='post', maxlen=max_len)
train_right = pad_sequences(train_right_int, padding='post', maxlen=max_len)

cv_left = pad_sequences(cv_left_int, padding='post', maxlen=max_len)
cv_right = pad_sequences(cv_right_int, padding='post', maxlen=max_len)

test_left = pad_sequences(test_left_int, padding='post', maxlen=max_len)
test_right = pad_sequences(test_right_int, padding='post', maxlen=max_len)

In [7]:
left_input = layers.Input(shape=(max_len,))
right_input = layers.Input(shape=(max_len,))

left_emd = embedded_layer(left_input)
right_emd = embedded_layer(right_input)

lstm = layers.LSTM(50, return_sequences=True)

left_lstm = lstm(left_emd)
right_lstm = lstm(right_emd)

left_lstm = layers.Dropout(0.2)(left_lstm)
right_lstm = layers.Dropout(0.2)(right_lstm)

left_attention = SeqSelfAttention(attention_activation='tanh')(left_lstm)
right_attention = SeqSelfAttention(attention_activation='tanh')(right_lstm)

left_f = layers.Flatten()(left_attention)
right_f = layers.Flatten()(right_attention)

concat = layers.concatenate([left_f, right_f], axis=-1)

concat = layers.Dense(128, activation='tanh')(concat)

output = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001))(concat)

optimizer = keras.optimizers.Adam()

model = Model(inputs=[left_input, right_input], outputs=[output])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 69)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 69)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 69, 300)      3849300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 69, 50)       70200       embedding_1[0][0]                
          

In [8]:
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['accuracy'])

from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',mode='min', verbose=1, patience=3, restore_best_weights=True)

model.fit([train_left, train_right], [train_label], batch_size=64, epochs=100, validation_data=([cv_left, cv_right], [cv_label]), callbacks=[es])

Train on 8800 samples, validate on 1100 samples
Epoch 1/100
8800/8800 [==============================] - 28s 3ms/step - loss: 0.2077 - acc: 0.6769 - val_loss: 0.1906 - val_acc: 0.6982
Epoch 2/100
8800/8800 [==============================] - 26s 3ms/step - loss: 0.1798 - acc: 0.7360 - val_loss: 0.1809 - val_acc: 0.7273
Epoch 3/100
8800/8800 [==============================] - 26s 3ms/step - loss: 0.1605 - acc: 0.7712 - val_loss: 0.1753 - val_acc: 0.7364
Epoch 4/100
8800/8800 [==============================] - 26s 3ms/step - loss: 0.1469 - acc: 0.7964 - val_loss: 0.1661 - val_acc: 0.7773
Epoch 5/100
8800/8800 [==============================] - 27s 3ms/step - loss: 0.1288 - acc: 0.8280 - val_loss: 0.1671 - val_acc: 0.7818
Epoch 6/100
8800/8800 [==============================] - 25s 3ms/step - loss: 0.1135 - acc: 0.8511 - val_loss: 0.1578 - val_acc: 0.7845
Epoch 7/100
8800/8800 [==============================] - 27s 3ms/step - loss: 0.0989 - acc: 0.8743 - val_loss: 0.1506 - val_acc: 0.7900


In [9]:
evaluation = model.evaluate([test_left, test_right], [test_label])

print('Accuracy: '+str(evaluation[1]))
print('Loss: '+str(evaluation[0]))

1100/1100 [==============================] - 4s 4ms/step
Accuracy: 0.8445454545454546
Loss: 0.12239359594204209
